In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action="ignore")

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

import tensorflow as tf

import warnings
warnings.filterwarnings(action="ignore")


In [ ]:
# 사용할 변수

# 식수 인원, 요일별 평균, 월평균?
# 메뉴
# 요일
# 휴일여부
# 날씨

In [2]:
# 데이터 준비
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [3]:
# 컬럼 정리
train.rename(columns={'본사정원수':'근무자수', '본사휴가자수':'휴가자수', '본사출장자수':'출장자수',
                      '본사시간외근무명령서승인건수':'야근자수', '현본사소속재택근무자수':'재택근무'}, inplace=True)

test.rename(columns={'본사정원수':'근무자수', '본사휴가자수':'휴가자수', '본사출장자수':'출장자수', 
            '본사시간외근무명령서승인건수':'야근자수', '현본사소속재택근무자수':'재택근무'}, inplace=True)

In [6]:
train.head(3)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      1205 non-null   object 
 1   요일      1205 non-null   object 
 2   근무자수    1205 non-null   int64  
 3   휴가자수    1205 non-null   int64  
 4   출장자수    1205 non-null   int64  
 5   야근자수    1205 non-null   int64  
 6   재택근무    1205 non-null   float64
 7   조식메뉴    1205 non-null   object 
 8   중식메뉴    1205 non-null   object 
 9   석식메뉴    1205 non-null   object 
 10  중식계     1205 non-null   float64
 11  석식계     1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [19]:
# 일자를 datetime 형식으로 변환

train['일자'] = pd.to_datetime(train['일자'])

In [20]:
train.head(2)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      1205 non-null   datetime64[ns]
 1   요일      1205 non-null   object        
 2   근무자수    1205 non-null   int64         
 3   휴가자수    1205 non-null   int64         
 4   출장자수    1205 non-null   int64         
 5   야근자수    1205 non-null   int64         
 6   재택근무    1205 non-null   float64       
 7   조식메뉴    1205 non-null   object        
 8   중식메뉴    1205 non-null   object        
 9   석식메뉴    1205 non-null   object        
 10  중식계     1205 non-null   float64       
 11  석식계     1205 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(4)
memory usage: 113.1+ KB


In [21]:
train.describe()

,근무자수,휴가자수,출장자수,야근자수,재택근무,중식계,석식계
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,2807.815768,157.913693,241.142739,274.117012,43.506224,890.334440,461.772614
std,171.264404,144.190572,43.532298,246.239651,109.937400,209.505057,139.179202
min,2601.000000,23.000000,41.000000,0.000000,0.000000,296.000000,0.000000
25%,2645.000000,71.000000,217.000000,4.000000,0.000000,758.000000,406.000000
50%,2760.000000,105.000000,245.000000,299.000000,0.000000,879.000000,483.000000
75%,2962.000000,185.000000,272.000000,452.000000,0.000000,1032.000000,545.000000
max,3305.000000,1224.000000,378.000000,1044.000000,533.000000,1459.000000,905.000000


In [38]:
# 요일별 평균

lunch_mean = pd.pivot_table(train, values = '중식계', index=['요일'], aggfunc = np.mean).reset_index()
lunch_mean = lunch_mean.sort_values('중식계', ascending=False)
lunch_mean = lunch_mean.reset_index(drop=True)
lunch_mean

,요일,중식계
0,월,1144.331950
1,화,925.620833
2,수,905.213389
3,목,823.991803
4,금,653.609959


In [39]:
# 요일 원핫 인코딩

df = pd.DataFrame({'월', '화', '수', '목', '금'})
pd.get_dummies(df)

,0_금,0_목,0_수,0_월,0_화
0,0,0,0,0,1
1,0,1,0,0,0
2,0,0,0,1,0
3,1,0,0,0,0
4,0,0,1,0,0


In [ ]:
# 월평균?

In [ ]:
# 날씨

# 여름철 폭염 : 기상청의 폭염주의보는 일 최고기온이 33도 이상인 상태가 2일정도
# 지속될 것으로 예상할 때

# 불쾌지수 
#  DI＝9/5(기온)－0.55(1－상대습도)(9/5(기온)－26)＋32 
# 현재 기상청에서 제공하고 있지 않음.

# 체감온도
# 체감온도 = 13.12 + 0.6215T - 11.37 V0.16 + 0.3965 V0.16T
# T : 기온, V : 풍속


In [ ]:
# 매해 같은 메뉴
# ex) 초복 삼계탕

# 이벤트 여부?

In [ ]:
# 휴일 여부

In [ ]:
# 연휴 여부

In [ ]:
# 코로나로 인한 재택근무

In [ ]:
# 연말

In [ ]:
# 메뉴

# open api 활용해서 메뉴 분류